<a href="https://colab.research.google.com/github/dpnolan/taba/blob/main/TABA_Bank_data_logic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part B
# Bank telesales data analysis using Logisitic Regression

In [4]:
# Start a new R notebook in Jupyter using URL https://colab.research.google.com/notebook#create=true&language=r 

# Install R and libraries 

The input file gets deleted whenever a runtime ends, so upload the file __Bank.csv__ through the upload icon (page plus up arrow) on the left hand column into the __sample_data__ directory

In [8]:
getwd()

[1] "/content"

In [18]:
setwd("/content/")

In [10]:
list.files()


[1] "Bank.csv"    "sample_data"

In [21]:
bank=read.csv('Bank.csv',header=TRUE, sep=",", row.names=NULL)
bank=data.frame(bank)

In [22]:
summary(bank)

  row.names          f.employed          married           secondary        
 Length:32471       Length:32471       Length:32471       Length:32471      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                                                                            
      no                X1523            no.1               yes           
 Length:32471       Min.   : -8019   Length:32471       Length:32471      
 Class :character   1st Qu.:    83   Class :character   Class :character  
 Mode  :character   Median :   478   Mode  :character   Mode  :character  
                    Mean   :  1433                                        
                    3rd Qu.:  1509                                        
           